In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


c:\Users\gaomi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))
def flatten_name(prefix, src_names):
    ret = []
    for c in src_names:
        if c[0] in ['time_id', 'stock_id']:
            ret.append(c[0])
        else:
            ret.append('.'.join([prefix] + list(c)))
    return ret

In [15]:
def preprocessor_book(stock_id):
        
        book_dtypes = {
            'stock_id':np.uint16, 
            'time_id': np.uint16,
            'seconds_in_bucket': np.uint16,
            'bid_price1': np.float32,
            'ask_price1': np.float32,
            'bid_price2': np.float32,
            'ask_price2': np.float32,
            'bid_size1': np.uint32,
            'ask_size1': np.uint32,
            'bid_size2': np.uint32,
            'ask_size2': np.uint32,
        }

        dir_path=f"individual_book_train/stock_{stock_id}.csv"
        data=pd.read_csv(dir_path)
        for column, dtype in book_dtypes.items():
            data[column] = data[column].astype(dtype)
        

        return data

In [25]:
def feature_generator(stock_id,forecast_period=60):
    
    def difference(series):
        return np.max(series)-np.min(series)
    
    data=preprocessor_book(stock_id=stock_id)
    data['wap1'] =(data['bid_price1'] * data['ask_size1'] + data['ask_price1'] * data['bid_size1'])/(data['bid_size1'] + data['ask_size1'])
    data['log_return1'] = data.groupby('time_id')['wap1'].transform(log_return)
    data['wap2'] = (data['bid_price2'] * data['ask_size2'] + data['ask_price2'] * data['bid_size2'])/(data['bid_size2'] + data['ask_size2'])
    data['log_return2'] = data.groupby('time_id')['wap2'].transform(log_return)

    data['wap_balance'] = abs(data['wap1'] - data['wap2'])
    data["BidAskSpread"]=(data['ask_price1'] / data['bid_price1'])-1
    
    data['bid_spread'] = data['bid_price1'] - data['bid_price2']
    data['ask_spread'] = data['ask_price1'] - data['ask_price2']
    data['total_volume'] = (data['ask_size1'] + data['ask_size2']) + (data['bid_size1'] + data['bid_size2'])
    data['volume_imbalance'] = abs((data['ask_size1'] + data['ask_size2']) - (data['bid_size1'] + data['bid_size2']))
    data['log_return_ask1'] = data.groupby(['time_id'])['ask_price1'].transform(log_return)
    data['log_return_ask2'] = data.groupby(['time_id'])['ask_price2'].transform(log_return)
    data['log_return_bid1'] = data.groupby(['time_id'])['bid_price1'].transform(log_return)
    data['log_return_bid2'] = data.groupby(['time_id'])['bid_price2'].transform(log_return)

    data_test=data[data["seconds_in_bucket"]>=600-forecast_period]
    data_train=data[data["seconds_in_bucket"]<600-forecast_period]
    #dict for aggregate
    create_feature_dict = {
    'seconds_in_bucket': ['count'],
    'wap1': [np.nanmean, np.nanstd,difference],
    'wap2': [np.nanmean, np.nanstd,difference],
    'log_return1': [np.nansum, realized_volatility],
    'log_return2': [np.nansum, realized_volatility],
    'log_return_ask1': [np.nansum, realized_volatility],
    'log_return_ask2': [np.nansum, realized_volatility],
    'log_return_bid1': [np.nansum, realized_volatility],
    'log_return_bid2': [np.nansum, realized_volatility],
    'wap_balance': [np.nanmean, np.nanstd],
    'BidAskSpread': [np.nanmax, np.nanmean,difference],
    'bid_spread': [np.nanmax, np.nanmean,difference],
    'ask_spread': [np.nanmax, np.nanmean,difference],
    'total_volume': [np.nansum, np.nanmean,difference],
    'volume_imbalance': [np.nansum, np.nanmean,np.nanmax]
}


    df_feature = pd.DataFrame(data_train.groupby(['time_id'],sort=False).agg(create_feature_dict)).reset_index()
    df_feature.columns = flatten_name('book', df_feature.columns)
    df_feature["missing_values"]=pd.DataFrame(data_train.groupby(['time_id'],sort=False).apply(lambda x: 540-len(x))).reset_index().iloc[:,1]
    for start_time in [250,350,450]:
        d = data_train[(data_train['seconds_in_bucket'] > start_time)].groupby('time_id').agg(create_feature_dict).reset_index(drop=False)
        d.columns = flatten_name(f'book_{start_time/60}to_9', d.columns)
        df_feature = pd.merge(df_feature, d, on='time_id', how='left')
    create_target_dict = {
        'log_return1':[realized_volatility],
            }
    target = pd.DataFrame(data_test.dropna(subset=["log_return1"]).groupby(['time_id'],sort=False).agg(create_target_dict)).reset_index()
    target.columns = flatten_name('book', target.columns)
    df_feature = pd.merge(df_feature,target,how='left',left_on='time_id',right_on=f'time_id')
    df_feature.rename(columns={'book.log_return1.realized_volatility_y': 'target'}, inplace=True)
    df_feature["stock_id"]=stock_id
    return df_feature

In [26]:
df_feat=feature_generator(1)
df_feat

,time_id,book.seconds_in_bucket.count,book.wap1.nanmean,book.wap1.nanstd,book.wap1.difference,book.wap2.nanmean,book.wap2.nanstd,book.wap2.difference,book.log_return1.nansum,book.log_return1.realized_volatility_x,book.log_return2.nansum,book.log_return2.realized_volatility,book.log_return_ask1.nansum,book.log_return_ask1.realized_volatility,book.log_return_ask2.nansum,book.log_return_ask2.realized_volatility,book.log_return_bid1.nansum,book.log_return_bid1.realized_volatility,book.log_return_bid2.nansum,book.log_return_bid2.realized_volatility,book.wap_balance.nanmean,book.wap_balance.nanstd,book.BidAskSpread.nanmax,book.BidAskSpread.nanmean,book.BidAskSpread.difference,book.bid_spread.nanmax,book.bid_spread.nanmean,book.bid_spread.difference,book.ask_spread.nanmax,book.ask_spread.nanmean,book.ask_spread.difference,book.total_volume.nansum,book.total_volume.nanmean,book.total_volume.difference,book.volume_imbalance.nansum,book.volume_imbalance.nanmean,book.volume_imbalance.nanmax,missing_values,book_4.166666666666667to_9.seconds_in_bucket.count,book_4.166666666666667to_9.wap1.nanmean,book_4.166666666666667to_9.wap1.nanstd,book_4.166666666666667to_9.wap1.difference,book_4.166666666666667to_9.wap2.nanmean,book_4.166666666666667to_9.wap2.nanstd,book_4.166666666666667to_9.wap2.difference,book_4.166666666666667to_9.log_return1.nansum,book_4.166666666666667to_9.log_return1.realized_volatility,book_4.166666666666667to_9.log_return2.nansum,book_4.166666666666667to_9.log_return2.realized_volatility,book_4.166666666666667to_9.log_return_ask1.nansum,book_4.166666666666667to_9.log_return_ask1.realized_volatility,book_4.166666666666667to_9.log_return_ask2.nansum,book_4.166666666666667to_9.log_return_ask2.realized_volatility,book_4.166666666666667to_9.log_return_bid1.nansum,book_4.166666666666667to_9.log_return_bid1.realized_volatility,book_4.166666666666667to_9.log_return_bid2.nansum,book_4.166666666666667to_9.log_return_bid2.realized_volatility,book_4.166666666666667to_9.wap_balance.nanmean,book_4.166666666666667to_9.wap_balance.nanstd,book_4.166666666666667to_9.BidAskSpread.nanmax,book_4.166666666666667to_9.BidAskSpread.nanmean,book_4.166666666666667to_9.BidAskSpread.difference,book_4.166666666666667to_9.bid_spread.nanmax,book_4.166666666666667to_9.bid_spread.nanmean,book_4.166666666666667to_9.bid_spread.difference,book_4.166666666666667to_9.ask_spread.nanmax,book_4.166666666666667to_9.ask_spread.nanmean,book_4.166666666666667to_9.ask_spread.difference,book_4.166666666666667to_9.total_volume.nansum,book_4.166666666666667to_9.total_volume.nanmean,book_4.166666666666667to_9.total_volume.difference,book_4.166666666666667to_9.volume_imbalance.nansum,book_4.166666666666667to_9.volume_imbalance.nanmean,book_4.166666666666667to_9.volume_imbalance.nanmax,book_5.833333333333333to_9.seconds_in_bucket.count,book_5.833333333333333to_9.wap1.nanmean,book_5.833333333333333to_9.wap1.nanstd,book_5.833333333333333to_9.wap1.difference,book_5.833333333333333to_9.wap2.nanmean,book_5.833333333333333to_9.wap2.nanstd,book_5.833333333333333to_9.wap2.difference,book_5.833333333333333to_9.log_return1.nansum,book_5.833333333333333to_9.log_return1.realized_volatility,book_5.833333333333333to_9.log_return2.nansum,book_5.833333333333333to_9.log_return2.realized_volatility,book_5.833333333333333to_9.log_return_ask1.nansum,book_5.833333333333333to_9.log_return_ask1.realized_volatility,book_5.833333333333333to_9.log_return_ask2.nansum,book_5.833333333333333to_9.log_return_ask2.realized_volatility,book_5.833333333333333to_9.log_return_bid1.nansum,book_5.833333333333333to_9.log_return_bid1.realized_volatility,book_5.833333333333333to_9.log_return_bid2.nansum,book_5.833333333333333to_9.log_return_bid2.realized_volatility,book_5.833333333333333to_9.wap_balance.nanmean,book_5.833333333333333to_9.wap_balance.nanstd,book_5.833333333333333to_9.BidAskSpread.nanmax,book_5.833333333333333to_9.BidAskSpread.nanmean,book_5.833333333333333to_9.BidAskSpread.difference,book_5.83333333333

In [27]:
df_feat["target"].isna().sum()

0

In [19]:
from tqdm import tqdm
non_tradable=[]
for i in tqdm(range(0,127)):
    try:
        dir_path=f"individual_book_train/stock_{i}.csv"
        data=pd.read_csv(dir_path)
    except:
        non_tradable.append(i)

tradable_id=[i for i in range(0,127) if i not in non_tradable]
tradable_id[:15]

100%|██████████| 127/127 [01:21<00:00,  1.55it/s]


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 46,
 47,
 48,
 50,
 51,
 52,
 53,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126]

In [28]:
def preprocessor(list_stock_ids):
    from joblib import Parallel, delayed 
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
    
            
        df_tmp = feature_generator(stock_id=stock_id)
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [29]:
final_df=preprocessor(tradable_id)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  3.8min finished


In [30]:
final_df.isna().sum().value_counts()

0     111
7      33
13      3
78      1
Name: count, dtype: int64

In [31]:
final_df

,time_id,book.seconds_in_bucket.count,book.wap1.nanmean,book.wap1.nanstd,book.wap1.difference,book.wap2.nanmean,book.wap2.nanstd,book.wap2.difference,book.log_return1.nansum,book.log_return1.realized_volatility_x,book.log_return2.nansum,book.log_return2.realized_volatility,book.log_return_ask1.nansum,book.log_return_ask1.realized_volatility,book.log_return_ask2.nansum,book.log_return_ask2.realized_volatility,book.log_return_bid1.nansum,book.log_return_bid1.realized_volatility,book.log_return_bid2.nansum,book.log_return_bid2.realized_volatility,book.wap_balance.nanmean,book.wap_balance.nanstd,book.BidAskSpread.nanmax,book.BidAskSpread.nanmean,book.BidAskSpread.difference,book.bid_spread.nanmax,book.bid_spread.nanmean,book.bid_spread.difference,book.ask_spread.nanmax,book.ask_spread.nanmean,book.ask_spread.difference,book.total_volume.nansum,book.total_volume.nanmean,book.total_volume.difference,book.volume_imbalance.nansum,book.volume_imbalance.nanmean,book.volume_imbalance.nanmax,missing_values,book_4.166666666666667to_9.seconds_in_bucket.count,book_4.166666666666667to_9.wap1.nanmean,book_4.166666666666667to_9.wap1.nanstd,book_4.166666666666667to_9.wap1.difference,book_4.166666666666667to_9.wap2.nanmean,book_4.166666666666667to_9.wap2.nanstd,book_4.166666666666667to_9.wap2.difference,book_4.166666666666667to_9.log_return1.nansum,book_4.166666666666667to_9.log_return1.realized_volatility,book_4.166666666666667to_9.log_return2.nansum,book_4.166666666666667to_9.log_return2.realized_volatility,book_4.166666666666667to_9.log_return_ask1.nansum,book_4.166666666666667to_9.log_return_ask1.realized_volatility,book_4.166666666666667to_9.log_return_ask2.nansum,book_4.166666666666667to_9.log_return_ask2.realized_volatility,book_4.166666666666667to_9.log_return_bid1.nansum,book_4.166666666666667to_9.log_return_bid1.realized_volatility,book_4.166666666666667to_9.log_return_bid2.nansum,book_4.166666666666667to_9.log_return_bid2.realized_volatility,book_4.166666666666667to_9.wap_balance.nanmean,book_4.166666666666667to_9.wap_balance.nanstd,book_4.166666666666667to_9.BidAskSpread.nanmax,book_4.166666666666667to_9.BidAskSpread.nanmean,book_4.166666666666667to_9.BidAskSpread.difference,book_4.166666666666667to_9.bid_spread.nanmax,book_4.166666666666667to_9.bid_spread.nanmean,book_4.166666666666667to_9.bid_spread.difference,book_4.166666666666667to_9.ask_spread.nanmax,book_4.166666666666667to_9.ask_spread.nanmean,book_4.166666666666667to_9.ask_spread.difference,book_4.166666666666667to_9.total_volume.nansum,book_4.166666666666667to_9.total_volume.nanmean,book_4.166666666666667to_9.total_volume.difference,book_4.166666666666667to_9.volume_imbalance.nansum,book_4.166666666666667to_9.volume_imbalance.nanmean,book_4.166666666666667to_9.volume_imbalance.nanmax,book_5.833333333333333to_9.seconds_in_bucket.count,book_5.833333333333333to_9.wap1.nanmean,book_5.833333333333333to_9.wap1.nanstd,book_5.833333333333333to_9.wap1.difference,book_5.833333333333333to_9.wap2.nanmean,book_5.833333333333333to_9.wap2.nanstd,book_5.833333333333333to_9.wap2.difference,book_5.833333333333333to_9.log_return1.nansum,book_5.833333333333333to_9.log_return1.realized_volatility,book_5.833333333333333to_9.log_return2.nansum,book_5.833333333333333to_9.log_return2.realized_volatility,book_5.833333333333333to_9.log_return_ask1.nansum,book_5.833333333333333to_9.log_return_ask1.realized_volatility,book_5.833333333333333to_9.log_return_ask2.nansum,book_5.833333333333333to_9.log_return_ask2.realized_volatility,book_5.833333333333333to_9.log_return_bid1.nansum,book_5.833333333333333to_9.log_return_bid1.realized_volatility,book_5.833333333333333to_9.log_return_bid2.nansum,book_5.833333333333333to_9.log_return_bid2.realized_volatility,book_5.833333333333333to_9.wap_balance.nanmean,book_5.833333333333333to_9.wap_balance.nanstd,book_5.833333333333333to_9.BidAskSpread.nanmax,book_5.833333333333333to_9.BidAskSpread.nanmean,book_5.833333333333333to_9.BidAskSpread.difference,book_5.83333333333

In [14]:
final_df.to_csv("merge_regression_data.csv")

In [24]:
final_df.to_pickle("merge_regression_data.pkl")